In [1]:
import numpy as np
import os
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import torch.utils.data as datatorch
import torch.nn as nn
import pandas as pd
import torch
from PIL import Image
import cv2
import torch.backends.cudnn as cudnn
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import torch.nn.functional as F

In [2]:
train_triplets = np.loadtxt('train_triplets.txt', dtype= 'str')
test_triplets = np.loadtxt('test_triplets.txt', dtype= 'str')

In [3]:
print(test_triplets.shape)

(59544, 3)


In [4]:
print(train_triplets.shape)
#print()
train_triplets , val_triplets = train_test_split(train_triplets, test_size = 0.1)
print(train_triplets.shape, val_triplets.shape)
half_index = np.int64((val_triplets.shape[0]-val_triplets.shape[0]%2)/2)
print(half_index)
val_labels = np.int64(np.ones((val_triplets.shape[0],)))
print(val_labels.shape)
val_triplets[half_index:, 1], val_triplets[half_index:, 2] = val_triplets[half_index:, 2], val_triplets[half_index:, 1].copy()
val_labels[half_index:] = np.int64(0)

(59515, 3)
(53563, 3) (5952, 3)
2976
(5952,)


In [5]:
train_dir = 'food/food'
train_files = os.listdir(train_dir)
test_files = os.listdir(train_dir)


class ImageTriplesSet(Dataset):
    def __init__(self , file_array, dir, mode='train', transform = None,labels =None):
        self.triple_list = list(map(tuple, file_array))
        self.mode = mode
        self.labels = labels
        self.dir = dir
        self.transform = transform
        
    def __len__(self):
        return len(self.triple_list)
    
    def __getitem__(self,idx):
        img1 = Image.open(os.path.join(self.dir, self.triple_list[idx][0] + '.jpg'))
        img2 = Image.open(os.path.join(self.dir, self.triple_list[idx][1] + '.jpg'))
        img3 = Image.open(os.path.join(self.dir, self.triple_list[idx][2] + '.jpg'))
        
        
        if self.transform is not None:
            img1 = self.transform(img1).numpy()
            img2 = self.transform(img2).numpy()
            img3 = self.transform(img3).numpy()
        if self.labels is None:
            return img1, img2, img3
        else:
            return img1, img2, img3, self.labels[idx]
            
        #concat_img = cv2.hconcat([img1, img2, img3]).astype('float32')
        #if self.mode == 'train':
            #label = self.labels[idx]
            #return concat_img , label
            
        #else:
            #return concat_img, int(self.triple_list[idx][:-4])
        
#data_transform = transforms.Compose([
  #  transforms.Resize(350,240),
  #  transforms.CenterCrop(240),
  #  transforms.ToTensor()
#])

data_transform = transforms.Compose([
        transforms.Resize((224,244)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

train_dataset = ImageTriplesSet(train_triplets, train_dir, transform = data_transform, labels = None)
val_dataset = ImageTriplesSet(val_triplets, train_dir, transform= data_transform, labels = None)
test_dataset = ImageTriplesSet(test_triplets, train_dir, mode="test" ,transform = data_transform,labels = None)

In [6]:
model = torch.hub.load('pytorch/vision', 'resnet34', pretrained=False)

Using cache found in C:\Users\user/.cache\torch\hub\pytorch_vision_master


model = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3,3)),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d((2,2)),
            torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3)),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d((2,2)),
            torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3)),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d((2,2)),
            torch.nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3,3)),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d((2,2)),
            nn.Flatten(),
            torch.nn.Linear(32768, 2048),
            )

In [7]:
learning_rate = 0.001
batch_size = 16
epochs = 4
logstep = int(10000 // batch_size)
margin = 0.01

train_loader = datatorch.DataLoader(dataset=train_dataset, 
                         shuffle=True, 
                         batch_size=batch_size)

val_loader = datatorch.DataLoader(dataset=val_dataset, shuffle = False, batch_size= batch_size)

#test_loader = datatorch.DataLoader(dataset=test_dataset, shuffle = False, batch_size= batch_size)



#model.fc = nn.Sequential(nn.Linear(model.fc.in_features,512),
                                  #nn.ReLU(),
                                  #nn.Dropout(),
                                  #nn.Linear(512, 2))
            
#model.fc = nn.Sequential(nn.Linear(model.fc.in_features,1024),
                                  #nn.ReLU(),
                                  #nn.Linear(1024, 2048))
        
model.fc = nn.Sequential(nn.Linear(model.fc.in_features,512),
                                  #nn.ReLU(),
                                  nn.Linear(512, 2048))      
        


#net = TripletNet(resnet101())
           
            

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cuda:0")
model =model.to(device)
#net = torch.nn.DataParallel(net).cuda()
#cudnn.benchmark = True
 #create optimizer
criterion = nn.TripletMarginLoss(margin=margin, p=2)
#criterion = nn.MarginRankingLoss(margin = 0.01)
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate,momentum=0.9,weight_decay=1e-5,nesterov=True)

#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='min',factor=0.1,patience=10,verbose=True)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[500,1000,1500], gamma=0.5)

training_loss_vec = []
training_accuracy_vec = []
val_loss_vec = []
val_f1_score = []
    
#criterion = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[500,1000,1500], gamma=0.5)

    
    
    

# loop over epochs

for e in range(epochs):
    training_loss = 0.
    training_accuracy = 0.
    
    model.train()
    for idx, (data1, data2, data3) in enumerate(train_loader):
    #for idx, (img,label) in enumerate(train_loader):
        data1, data2, data3 = data1.cuda(), data2.cuda(), data3.cuda()
        #img, label = img.cuda(), label.cuda()
        #embedded_a, embedded_p, embedded_n = net(data1, data2, data3)
        embedded_a, embedded_p, embedded_n = model(data1), model(data2), model(data3)
        
        
        dist_a = F.pairwise_distance(embedded_a, embedded_n, 2)
        dist_b = F.pairwise_distance(embedded_a, embedded_p, 2)
        
        # compute predictions using model
        #y_pred =  model(img)
        # compute loss
        #target = torch.FloatTensor(dist_a.size()).fill_(1)
        #target = target.cuda()
        #rank_loss = criterion(dist_a,dist_b,target)
        loss_embedd = embedded_a.norm(2) + embedded_n.norm(2) + embedded_p.norm(2)
        #loss = rank_loss# + 0.001*loss_embedd
        loss = criterion(embedded_a, embedded_p, embedded_n)+ 0.001*loss_embedd #tripletmarginloss
        #loss = criterion(y_pred,label) 
        # call optimizer.zero_grad()
        optimizer.zero_grad()
        # run backward method
        loss.backward()
        # run optimizer step
        optimizer.step()
        #scheduler.step()  ######
        # logging (optional)
        
        training_loss += loss.item()
        pred = (dist_a - dist_b).cpu().data
        training_accuracy  += torch.mean(((pred >0)*1).float()).item()
        
        #y_pred_idx = torch.max(y_pred.detach().cpu(),dim=1)[1]
        #training_accuracy += torch.mean((y_pred_idx == label.cpu()).float()).item()
        
        if (idx+1) % logstep == 0: 
            training_loss_vec.append(training_loss/logstep)
            training_accuracy_vec.append(training_accuracy/logstep)
            print('training loss: ', training_loss/logstep, '\n', 'training accuracy: ', training_accuracy/logstep)
            training_loss, training_accuracy = 0.,0.
            
    val_labels_pred = []
    model.eval()
    for idx, (data1, data2, data3) in enumerate(val_loader):
        data1, data2, data3 = data1.cuda(), data2.cuda(), data3.cuda()
        embedded_a, embedded_p, embedded_n = model(data1), model(data2), model(data3)

        dist_a = F.pairwise_distance(embedded_a, embedded_n, 2)
        dist_b = F.pairwise_distance(embedded_a, embedded_p, 2)
        diff = (dist_a - dist_b).cpu().data

        pred = (diff > 0).int().tolist()
        val_labels_pred += pred

    f1 = f1_score(val_labels_pred, val_labels)
    acc = accuracy_score(val_labels_pred, val_labels)
    val_f1_score.append(f1)
    print('Epoch :',e,' Current Validation F1 Score: ', f1, 'Current Val Acc:', acc)
    
   
    

training loss:  0.1503482230246067 
 training accuracy:  0.5542
training loss:  0.10152836930453778 
 training accuracy:  0.5479
training loss:  0.08855695036351681 
 training accuracy:  0.5429
training loss:  0.08246005915403366 
 training accuracy:  0.5499
training loss:  0.07232446353435516 
 training accuracy:  0.5652
Epoch : 0  Current Validation F1 Score:  0.5735368103303706 Current Val Acc: 0.5727486559139785
training loss:  0.06065049070417881 
 training accuracy:  0.5912
training loss:  0.05582641283869744 
 training accuracy:  0.5924
training loss:  0.049184988567233086 
 training accuracy:  0.6073
training loss:  0.043950296711921695 
 training accuracy:  0.6206
training loss:  0.03887475598007441 
 training accuracy:  0.6398
Epoch : 1  Current Validation F1 Score:  0.6554031448645031 Current Val Acc: 0.6538978494623656
training loss:  0.03128903599530458 
 training accuracy:  0.6845
training loss:  0.028373077622056007 
 training accuracy:  0.6945
training loss:  0.02638618

In [ ]:
## evaluation


#Epoch : 2  Current Validation F1 Score:  0.734122353725621 Current Val Acc: 0.7320228494623656
print(val_f1_score)
#Epoch : 3  Current F1 Score:  0.8208248816768087
# 0.5891517599538373

In [ ]:
#for i in range(0, len(val_labels_pred)): 
    #val_labels_pred[i] = int(val_labels_pred[i]) 
#f1 = f1_score(val_labels_pred, val_labels)

print(len(val_labels_pred,))

In [ ]:
test_loader = datatorch.DataLoader(dataset=test_dataset, shuffle = False, batch_size= 8)

model.eval()
for idx, (data1, data2, data3) in enumerate(test_loader):
    data1, data2, data3 = data1.cuda(), data2.cuda(), data3.cuda()
    embedded_a, embedded_p, embedded_n = model(data1), model(data2), model(data3)
    dist_a = F.pairwise_distance(embedded_a, embedded_n, 2)
    dist_b = F.pairwise_distance(embedded_a, embedded_p, 2)
    diff = (dist_a - dist_b)
    if diff[diff > 0.015].size()[0]!=0:
        model.train()
        target = torch.FloatTensor(dist_a[diff > 0.015].size()).fill_(1)
        target = target.cuda()
        rank_loss = criterion(dist_a[diff > 0.015],dist_b[diff > 0.015],target)
        #loss_embedd = embedded_a.norm(2) + embedded_n.norm(2) + embedded_p.norm(2)
        loss = rank_loss# + 0.001*loss_embedd
        #loss = criterion(y_pred,label)
        # call optimizer.zero_grad()
        optimizer.zero_grad()
        # run backward method
        loss.backward()
        # run optimizer step
        optimizer.step()
        model.eval()

In [ ]:
test_triplets_pred = []
model.eval()
for idx, (data1, data2, data3) in enumerate(test_loader):
    data1, data2, data3 = data1.cuda(), data2.cuda(), data3.cuda()
    embedded_a, embedded_p, embedded_n = model(data1), model(data2), model(data3)
    dist_a = F.pairwise_distance(embedded_a, embedded_n, 2)
    dist_b = F.pairwise_distance(embedded_a, embedded_p, 2)
    diff = (dist_a - dist_b)
    diff = diff.cpu().data
    #print(diff[diff > 0.015], len(diff[diff > 0.15].size()))#embedded_a[np.abs(diff)>0.015,:])
    pred = (diff > margin).int().tolist()
    test_triplets_pred += pred
    

In [ ]:
print(len(test_triplets_pred))

In [ ]:
with open('submission4_Ketzel.txt', 'w') as f:
    for item in test_triplets_pred:
        f.write(str(item) + '\n')